In [1]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
import pathlib
data_dir = r"DataSet_Path"
data_dir = pathlib.Path(data_dir)
print(f"Dataset directory: {data_dir}")

Dataset directory: C:\Users\Deepak\Documents\major projecct(rean)\Source code (VS)\balanced data set (resized)


In [3]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

3989


In [4]:
batch_size = 32
img_height = 224
img_width = 224

In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4000 files belonging to 4 classes.
Using 3200 files for training.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 4000 files belonging to 4 classes.
Using 800 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['Mild Anemia', 'Moderate Anemia', 'No Anemia', 'Severe Anemia']


In [8]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 224, 224, 3)
(32,)


In [9]:
normalization_layer = layers.Rescaling(1./255)

In [10]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [ ]:
dataset_path = r"DataSet_Path"
import os
# Get class-wise image count
class_counts = {}
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    if os.path.isdir(class_path):  # Ensure it's a folder
        class_counts[class_name] = len(os.listdir(class_path))

# Print the count for each class
for class_name, count in class_counts.items():
    print(f"Class: {class_name} -> Images: {count}")

Class: Mild Anemia -> Images: 1000
Class: Moderate Anemia -> Images: 1000
Class: No Anemia -> Images: 1000
Class: Severe Anemia -> Images: 1000


#Model 1 (Best Model Transfer learning)

In [ ]:


from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

base_model=MobileNetV3Large(weights='imagenet',include_top=False, input_shape=(224,224,3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.3)(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.3)(x)

preds=Dense(4,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_1 (Rescaling)     (None, 224, 224, 3)          0         ['input_1[0][0]']             
                                                                                                  
 Conv (Conv2D)               (None, 112, 112, 16)         432       ['rescaling_1[0][0]']         
                                                                                                  
 Conv/BatchNorm (BatchNorma  (None, 112, 112, 16)         64        ['Conv[0][0]']                
 lization)                                                                                    

In [18]:
model.compile(optimizer=Adam(learning_rate=0.0001),
               loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [19]:
epochs=15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)


# Save the trained model as an .h5 file
# model.save("model.h5")

# print("Model saved successfully as model.h5")

# Before saving
print(model.get_weights()[0])  # Print first layer weights

# Save model
model.save("model.h5")

# Load model again
from tensorflow.keras.models import load_model
loaded_model = load_model("model.h5")

# After loading
print(loaded_model.get_weights()[0])  # Compare weights


Epoch 1/15
100/100 [==============================] - 25s 224ms/step - loss: 3.6677 - accuracy: 0.5184 - val_loss: 3.1678 - val_accuracy: 0.7075
Epoch 2/15
100/100 [==============================] - 22s 218ms/step - loss: 3.0945 - accuracy: 0.6950 - val_loss: 2.8210 - val_accuracy: 0.7700
Epoch 3/15
100/100 [==============================] - 22s 217ms/step - loss: 2.7955 - accuracy: 0.7541 - val_loss: 2.5746 - val_accuracy: 0.8225
Epoch 4/15
100/100 [==============================] - 22s 218ms/step - loss: 2.5425 - accuracy: 0.8144 - val_loss: 2.3840 - val_accuracy: 0.8675
Epoch 5/15
100/100 [==============================] - 22s 216ms/step - loss: 2.3561 - accuracy: 0.8541 - val_loss: 2.2676 - val_accuracy: 0.8687
Epoch 6/15
100/100 [==============================] - 22s 217ms/step - loss: 2.1811 - accuracy: 0.8944 - val_loss: 2.0925 - val_accuracy: 0.9162
Epoch 7/15
100/100 [==============================] - 22s 219ms/step - loss: 2.0615 - accuracy: 0.9184 - val_loss: 1.9788 - val_ac

C:\Users\Deepak\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[[[[ 4.74257290e-01  3.84327859e-01 -5.35630465e-01  3.55508886e-02
     2.88742006e-01  6.94768667e-01  8.48755240e-02 -4.37884361e-01
     3.74123193e-02  2.09846795e-01  7.10104853e-02  2.88548261e-01
     4.00461972e-01 -5.21113813e-01 -2.53070295e-01 -3.63211691e-01]
   [-2.07101926e-01 -7.34134853e-01 -1.04572845e+00 -2.03642268e-02
     4.34004098e-01 -4.76945043e-01 -4.75457720e-02 -8.79060984e-01
    -6.75275102e-02  2.86088586e-01  1.08445607e-01  6.58086061e-01
     2.90191799e-01 -9.57054377e-01  5.52441657e-01 -4.56653982e-01]
   [-2.07804918e-01  3.55969936e-01 -2.68229634e-01 -1.49971228e-02
     2.06070811e-01 -2.55605727e-01  9.15222242e-02 -2.33704612e-01
     6.88022422e-03  1.35320961e-01 -6.68843910e-02  1.93160713e-01
     4.47369069e-01 -2.48584956e-01 -3.21647286e-01 -2.26869017e-01]]

  [[ 7.06022382e-01  3.30924630e-01  4.77267355e-01  4.35036749e-01
    -6.19506538e-01  7.57351458e-01 -1.73452720e-01 -5.43356121e-01
    -7.23181572e-03  1.85976401e-01  1.9028

In [20]:
import numpy as np

# Check if weights are the same before and after saving/loading
original_weights = model.get_weights()[0]
loaded_weights = loaded_model.get_weights()[0]

if np.array_equal(original_weights, loaded_weights):
    print("Weights are identical after loading.")
else:
    print("Weights have changed after loading.")


Weights are identical after loading.


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:

from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam


base_model=MobileNetV3Large(weights='imagenet',include_top=False, input_shape=(224,224,3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)  # Add BatchNorm before activation
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)

x = Dense(256, kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)  # Add BatchNorm before activation
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)  # Add BatchNorm before activation
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

preds=Dense(4,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)

model.compile(optimizer=Adam(learning_rate=0.0001),
               loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
epochs=25
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()